<a href="https://colab.research.google.com/github/asyraffatha/Task-MachineLearning/blob/main/Week%2011/Dummy_Data_Classification_Asyraff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Step 1: Memuat data
file_path = '/content/drive/MyDrive/Week 11/heart.csv'
data = pd.read_csv(file_path)

In [4]:
# Menampilkan beberapa baris pertama
print("Data Overview:\n", data.head())
print("\nData Info:\n")
data.info()

Data Overview:
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   52    1   0       125   212    0        1      168      0      1.0      2   
1   53    1   0       140   203    1        0      155      1      3.1      0   
2   70    1   0       145   174    0        1      125      1      2.6      0   
3   61    1   0       148   203    0        1      161      0      0.0      2   
4   62    0   0       138   294    1        1      106      0      1.9      1   

   ca  thal  target  
0   2     3       0  
1   0     3       0  
2   0     3       0  
3   1     3       0  
4   3     2       0  

Data Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1025 entries, 0 to 1024
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1025 non-null   int64  
 1   sex       1025 non-null   int64  
 2   cp        1025 non-null   int64  
 3   trestbps  1025 non-null   int64  
 4   chol  

In [7]:
# Step 2: Preprocessing
# Memisahkan fitur dan target
X = data.drop(columns=['target'])
y = data['target']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# Membagi data ke dalam set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [9]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [10]:
# Step 3: Membuat kelas himpunan data kustom
class HeartDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Mmembuat dataset
dataset_train = HeartDataset(X_train_tensor, y_train_tensor)
dataset_test = HeartDataset(X_test_tensor, y_test_tensor)

In [11]:
# Step 4: Menentukan kelas model MLP
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_layers, activation_fn):
        super(MLPClassifier, self).__init__()
        layers = []
        in_features = input_size

        # Tambahkan lapisan tersembunyi
        for hidden_units in hidden_layers:
            layers.append(nn.Linear(in_features, hidden_units))
            layers.append(activation_fn)
            in_features = hidden_units

        # Tambahkan lapisan output
        layers.append(nn.Linear(in_features, 2))  # Binary classification (2 classes)
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [13]:
# Step 5: Tentukan fungsi pelatihan dan evaluasi
def train_and_evaluate_model(hidden_layers, activation_fn, epochs, learning_rate, batch_size):
    # Define model
    model = MLPClassifier(input_size=X_train.shape[1], hidden_layers=hidden_layers, activation_fn=activation_fn)

    # Kerugian dan pengoptimal
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Memeuat data
    train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

    # Latih
    for epoch in range(epochs):
        model.train()
        for features, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # Evaluasi
    model.eval()
    all_preds = []
    with torch.no_grad():
        for features, _ in test_loader:
            preds = model(features)
            all_preds.append(preds.argmax(dim=1).numpy())

    all_preds = np.concatenate(all_preds)
    accuracy = accuracy_score(y_test, all_preds)
    return accuracy

In [14]:
# Step 6: Menjalankan eksperimen dengan berbagai konfigurasi
activation_functions = {
    "ReLU": nn.ReLU(),
    "Sigmoid": nn.Sigmoid(),
    "Tanh": nn.Tanh(),
    "Softmax": nn.Softmax(dim=1),
    "Linear": nn.Identity()
}


In [18]:
hidden_layer_configs = [[4], [8, 4], [16, 8, 4]]
epochs_list = [1, 10, 25, 50, 100, 250]
learning_rates = [10, 1, 0.1, 0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64, 128, 256, 512]

results = []

In [21]:
for activation_name, activation_fn in activation_functions.items():
    for hidden_layers in hidden_layer_configs:
        for epochs in epochs_list:
            for lr in learning_rates:
                for batch_size in batch_sizes:
                    accuracy = train_and_evaluate_model(
                        hidden_layers=hidden_layers,
                        activation_fn=activation_fn,
                        epochs=epochs,
                        learning_rate=lr,
                        batch_size=batch_size
                    )
                    results.append({
                        "Activation": activation_name,
                        "Hidden Layers": hidden_layers,
                        "Epochs": epochs,
                        "Learning Rate": lr,
                        "Batch Size": batch_size,
                        "Accuracy": accuracy
                    })

# Mengonversi hasil ke DataFrame dan tampilkan
results_df = pd.DataFrame(results)
print("\nExperiment Results:\n", results_df.head())

# Simpan hasil eksperimen
results_df.to_csv('/content/drive/MyDrive/Week 11/experiment_results.csv', index=False)


Experiment Results:
   Activation Hidden Layers  Epochs  Learning Rate  Batch Size  Accuracy
0       ReLU           [4]       1           10.0          16  0.575610
1       ReLU           [4]       1           10.0          32  0.434146
2       ReLU           [4]       1           10.0          64  0.673171
3       ReLU           [4]       1           10.0         128  0.775610
4       ReLU           [4]       1           10.0         256  0.770732
